In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install dicomsdl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.1 MB/s eta 0:00:00a 0:00:01


# Converting images to png

In [6]:
import dicomsdl as dicoml
import cv2
import pydicom

from joblib import Parallel, delayed
import glob
import time
import numpy as np
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras

In [4]:
image_dir_dicomsdl = '/kaggle/working/png_file_dic/'
os.makedirs(image_dir_dicomsdl, exist_ok=True)
IMAGES_TO_PROCESS = 500
train_images = glob.glob("/kaggle/input/rsna-breast-cancer-detection/train_images/*/*.dcm")

def process(f, size=512, save_folder=None, extension="png"):    
    patient = f.split('/')[-2]
    image_name = f.split('/')[-1][:-4]
    dicom = dicoml.open(f)
    img = dicom.pixelData()
    img = (img - img.min()) / (img.max() - img.min())
    if dicom.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
        img = 1 - img
    image = (img * 255).astype(np.uint8)
    img = cv2.resize(image, (size, size))
    file_name = f'{save_folder}' + f"{patient}_{image_name}.{extension}"
    cv2.imwrite(file_name, img)

In [5]:
%%time
Parallel(n_jobs=4)(
    delayed(process)(f, size = 512, save_folder = image_dir_dicomsdl)
    for f in train_images[:IMAGES_TO_PROCESS]
)

CPU times: user 4.53 s, sys: 141 ms, total: 4.67 s
Wall time: 3min 56s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Preprocessing

In [4]:
import tensorflow as tf
from tensorflow import keras

In [9]:
path = '/kaggle/working/png_file_dic'
data = tf.data.Dataset.list_files(path + '/*.png')

In [10]:
def load_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image)
    return image

data = data.map(load_image)

In [12]:
data = data.batch(batch_size=100)
data = data.shuffle(buffer_size=1000)

In [14]:
def preprocess_image(image):
    image = tf.image.resize(image, (512, 512))
    image /= 255.0  # normalize to [0,1] range
    return image

data = data.map(preprocess_image)

In [15]:
X = data.unbatch()

In [16]:
X

<_UnbatchDataset shapes: (512, 512, None), types: tf.float32>